## 1. Setup and Configuration

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.cluster import KMeans

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 2. Import DatabaseManager SDK

In [ ]:
from src.database.database_utils import DatabaseManager

db = DatabaseManager()
print('Database connection established')

## 3. Basic Queries - Get Data as DataFrames

### 3.1 Query all accidents with filters

In [ ]:
# Get accidents from year 2022, 100 records
df_accidents = db.query_accidents(annee=2022, limit=100)
print(f'Shape: {df_accidents.shape}')
print('\nColumns:')
print(df_accidents.columns.tolist())
print('\nFirst rows:')
df_accidents.head()

### 3.2 Get accidents by specific commune

In [ ]:
# Paris commune code: 75056
df_paris = db.get_accidents_by_commune('75056', limit=50)
print(f'Accidents in Paris: {len(df_paris)}')
print(f'Date range: {df_paris.date_accident.min()} to {df_paris.date_accident.max()}')

### 3.3 Get accidents by region

In [ ]:
# Get accidents in Ile-de-France, year 2021
df_idf = db.get_accidents_by_region('Ile-de-France', annee=2021)
print(f'Accidents in Ile-de-France (2021): {len(df_idf)}')
print('\nAccidents by department:')
print(df_idf.code_dept.value_counts().head())

## 4. Aggregate Statistics

### 4.1 Temporal statistics (by day/hour)

In [ ]:
df_temps = db.get_stats_temporelles(annee=2022)
print(f'Temporal data shape: {df_temps.shape}')
print('\nStatistics by day of week:')
stats_jour = df_temps.groupby('jour_semaine')['nombre_accidents'].sum().sort_values(ascending=False)
print(stats_jour)

### 4.2 Most dangerous communes

In [ ]:
df_communes = db.get_stats_communes(limit=20)
print('Top 20 most dangerous communes:')
print(df_communes[['nom_com', 'nom_dept', 'nombre_accidents', 'nombre_deces', 'gravite_moyenne']].head(10))

### 4.3 Danger scores by commune

In [ ]:
df_danger = db.get_danger_scores(limit=30)
print('Top 10 communes by danger score:')
print(df_danger[['nom_com', 'score_danger', 'categorie_risque', 'nombre_accidents']].head(10))

## 5. Exploratory Data Analysis (EDA)

### 5.1 Basic statistics

In [ ]:
df_accidents = db.query_accidents(limit=1000)
print('Shape:', df_accidents.shape)
print('\nData types:')
print(df_accidents.dtypes)
print('\nMissing values:')
print(df_accidents.isnull().sum())

### 5.2 Numerical summary

In [ ]:
print(df_accidents.describe())

### 5.3 Correlation analysis

In [ ]:
# Select numeric columns
numeric_cols = df_accidents.select_dtypes(include=[np.number]).columns
corr_matrix = df_accidents[numeric_cols].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Matrix - Accidents Data')
plt.tight_layout()
plt.show()

## 6. Dimensionality Reduction (PCA)

In [ ]:
# Prepare data
df_numeric = df_accidents[numeric_cols].dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_numeric)

# Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f'Explained variance ratio: {pca.explained_variance_ratio_}')
print(f'Total variance explained: {pca.explained_variance_ratio_.sum():.2%}')

### Visualize PCA

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5, s=20)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.title('PCA - Accidents Data')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Factor Analysis

In [ ]:
# Apply Factor Analysis
fa = FactorAnalysis(n_components=2, random_state=42)
X_fa = fa.fit_transform(X_scaled)

# Loadings (importance of each original variable in each factor)
loadings = pd.DataFrame(
    fa.components_.T,
    columns=['Factor 1', 'Factor 2'],
    index=numeric_cols
)

print('Factor Loadings:')
print(loadings)

## 8. Clustering Analysis (K-Means)

In [ ]:
# Determine optimal number of clusters using elbow method
inertias = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 5))
plt.plot(k_range, inertias, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Apply K-Means with optimal k=3

In [ ]:
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

print(f'Cluster distribution:')
unique, counts = np.unique(clusters, return_counts=True)
for u, c in zip(unique, counts):
    print(f'  Cluster {u}: {c} records ({c/len(clusters)*100:.1f}%)')

### Visualize clusters in PCA space

In [ ]:
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.6, s=30)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.title(f'K-Means Clustering (k={optimal_k}) in PCA Space')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Visualization Examples

### 9.1 Accidents by day of week (Interactive Plotly)

In [ ]:
df_accidents = db.query_accidents(limit=5000)

acc_by_day = df_accidents.groupby('jour_semaine').size().reset_index(name='count')
fig = px.bar(acc_by_day, x='jour_semaine', y='count',
             title='Accidents by Day of Week',
             labels={'count': 'Number of Accidents', 'jour_semaine': 'Day of Week'})
fig.show()

### 9.2 Temporal trends (Interactive)

In [ ]:
df_temps_all = db.get_stats_temporelles()

# Aggregate by month
df_temps_all['year_month'] = pd.to_datetime(df_temps_all[['annee', 'mois']].assign(jour=1))
monthly = df_temps_all.groupby('year_month')['nombre_accidents'].sum().reset_index()

fig = px.line(monthly, x='year_month', y='nombre_accidents',
              title='Monthly Accident Trends',
              labels={'nombre_accidents': 'Number of Accidents', 'year_month': 'Date'})
fig.show()

### 9.3 Top communes by danger score (Interactive)

In [ ]:
df_danger = db.get_danger_scores(limit=15)

fig = px.bar(df_danger, x='score_danger', y='nom_com',
             orientation='h',
             color='score_danger',
             title='Top 15 Most Dangerous Communes by Risk Score',
             labels={'score_danger': 'Danger Score', 'nom_com': 'Commune'})
fig.update_layout(height=500)
fig.show()

## 10. Custom SQL Queries

For more advanced analyses, you can write custom SQL queries:

In [ ]:
# Example: Get accidents severity distribution by hour
custom_query = """
SELECT 
    heure,
    COUNT(*) as nombre,
    AVG(CASE WHEN gravite_max = 4 THEN 1 ELSE 0 END) as taux_mortalite
FROM accidents_schema.accidents
GROUP BY heure
ORDER BY heure
"""

df_custom = db.query_to_dataframe(custom_query)
print(df_custom)

## 11. Export Results

In [ ]:
# Export to CSV
df_accidents.to_csv('accidents_export.csv', index=False)
print('Exported to accidents_export.csv')

# Export to Excel
df_accidents.to_excel('accidents_export.xlsx', index=False)
print('Exported to accidents_export.xlsx')

## 12. Summary

This notebook demonstrates:

- **Data Retrieval**: Using DatabaseManager to fetch data as Pandas DataFrames
- **Exploratory Analysis**: Statistical summaries, correlations
- **Dimensionality Reduction**: PCA and Factor Analysis
- **Clustering**: K-Means and elbow method
- **Visualization**: Matplotlib, Seaborn, and Plotly
- **Custom Queries**: Writing SQL for specific analyses
- **Export**: Saving results to CSV/Excel

Feel free to adapt and extend this notebook for your own analyses!